<a href="https://colab.research.google.com/github/taddbackus/capstone/blob/main/Data_Cleanup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References:
https://geemap.org/notebooks/114_dynamic_world/


# Imports and Setup

In [1]:
!pip -q install --upgrade folium
!pip -q install geopandas
!pip -q install geojson
!pip -q install eeconvert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 114.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.9 MB/s eta 0:00:00


In [2]:
# reminder that if you are installing libraries in a Google Colab instance you will be prompted to restart your kernal

try:
    import geemap, ee
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print("package not found, installing w/ pip in Google Colab...")
        !pip install geemap
    else:
        print("package not found, installing w/ conda...")
        !conda install mamba -c conda-forge -y
        !mamba install geemap -c conda-forge -y
    import geemap, ee

package not found, installing w/ pip in Google Colab...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 103.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 101.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 65.3 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11124 sha256=a0ca9e9c909e8f

In [3]:
pip install eemont

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 kB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 28.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eemont: filename=eemont-0.3.6-py3-none-any.whl size=135743 sha256=7bec4edb258eef367a48783b60cf663db20215232fe8107cf8892fd5106a90d8
  Stored in directory: /root/.cache/pip/wheels/c2/79/2c/4fed17c3d3b466bbf4fe5872eec11f189147043b01152a4f75
  Created wheel for ee_extra: filename=ee_extra-0.0.15-py3-none-any.whl size=236757 sha256=f77690ee93542ace7d1ee5829de9e09bb234f9f1a0e21791a76bb8271ad1d2f8
  Stored in directory: /root/.cache/pip/wheels/29/96/0e/4e36b0dfd85e16867723df739294c0aa45a65b191adac4d959
Successfully built eemont ee_extra


In [4]:
# Standard imports
import requests
import json

import pandas as pd
import numpy as np
from PIL import Image

# Geospatial processing packages
import geopandas as gpd
import geojson

import shapely
from shapely.geometry import box

# Mapping and plotting libraries
import matplotlib.pyplot as plt
import matplotlib.colors as cl
import ee
import eeconvert as eec
import geemap
import folium
import eemont

import geopandas as gpd
from shapely.geometry import shape

### Mount Drive

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Authenticate Google Earth Engine
Make sure you have signed up for access to Google Earth Engine at https://signup.earthengine.google.com/#!/. Once your request has been approved, you should be able to access Google Earth Engine at https://code.earthengine.google.com/.

In [6]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=Optmik5WttEw6O4I4zN32CzgFJoZWcGEJtqu4x7_8LM&tc=pKNoOAwbDhO5hKkpB9YwZDHIsNI90OG920TjwQ7_ots&cc=bxrXFaP8873uR2nSSsSM9r4ErzqFCBP04LCCz4JPYvo

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AZEOvhWu8fPc8nZm3PncwJBu63PbSeV4nRuIerROd6ls3NxAEg0u3hQj14Q

Successfully saved authorization token.


In [54]:
# Read the GeoJSON file
mines = gpd.read_file('/content/drive/MyDrive/OSMRE_GEOMINE_GEOJSON.geojson')

In [55]:
mines

,OBJECTID,permittee,company,coalmine_op_status,mine_name,permit_id,msha_id,state_tribe_mine_id,national_id,coal_bed_names,...,mountaintop,steep_slope,highwall,auger,comment,contact,information_link,shape_Length,shape_Area,geometry
0,1,HIGGINS COAL CO INC,HIGGINS COAL CO INC,7,MINE #50,2404,NaN,NaN,TN2404,NaN,...,NaN,NaN,NaN,NaN,NaN,4,https://www.osmre.gov/,1220.769913,1.354382e+04,"MULTIPOLYGON (((-9515951.177 4212750.257, -951..."
1,2,MIDNIGHT COAL CO,MIDNIGHT COAL CO,7,MINE #1,2405,NaN,NaN,TN2405,NaN,...,NaN,NaN,NaN,NaN,NaN,4,https://www.osmre.gov/,860.404674,1.678249e+04,"MULTIPOLYGON (((-9516024.476 4212708.540, -951..."
2,3,META ELKHORN MINING CO INC,META ELKHORN MINING CO INC,7,AREA 11,2756,NaN,NaN,TN2756,NaN,...,NaN,NaN,NaN,NaN,NaN,4,https://www.osmre.gov/,7219.650616,1.689153e+06,"MULTIPOLYGON (((-9459511.615 4347543.957, -945..."
3,4,JORDAN BROTHERS COAL CO INC,JORDAN BROTHERS COAL CO INC,7,TIPPLE #1,2497,NaN,NaN,TN2497,NaN,...,NaN,NaN,NaN,NaN,NaN,4,https://www.osmre.gov/,688.042643,2.188255e+04,"MULTIPOLYGON (((-9372128.011 4337061.595, -937..."
4,5,DOUBLE Q INC,DOUBLE Q INC,7,ANGELA #1,2284023,NaN,NaN,TN2284023,NaN,...,NaN,NaN,NaN,NaN,NaN,4,https://www.osmre.gov/,7994.318732,8.636343e+04,"MULTIPOLYGON (((-9344712.695 4372890.512, -934..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25119,25120,Justice Coal Of Alabama LLC,Justice Coal Of Alabama LLC,1,Crane Central,P3946,0103433,P3946r1,ALP3946,NaN,...,NaN,NaN,NaN,NaN,State defined mine status: Expired. Reclamatio...,8,https://surface-mining.alabama.gov/,5370.984081,9.330482e+05,"MULTIPOLYGON (((-9540804.196 4143151.210, -954..."
25120,25121,Justice Coal Of Alabama LLC,Justice Coal Of Alabama LLC,1,Crane Central,P3946,0103433,P3946r1,ALP3946,NaN,...,NaN,NaN,NaN,NaN,State defined mine status: Expired. Reclamatio...,8,https://surface-mining.alabama.gov/,534.608228,1.452293e+04,"MULTIPOLYGON (((-9541424.447 4142622.814, -954..."
25121,25122,Justice Coal Of Alabama LLC,Justice Coal Of Alabama LLC,1,Crane Central,P3946,0103433,P3946r1,ALP3946,NaN,...,NaN,NaN,NaN,NaN,State defined mine status: Expired. Reclamatio...,8,https://surface-mining.alabama.gov/,507.253927,1.464629e+04,"MULTIPOLYGON (((-9540236.664 4142842.258, -954..."
25122,25123,Justice Coal Of Alabama LLC,Justice Coal Of Alabama LLC,1,Crane Central,P3946,0103433,P3946r1,ALP3946,NaN,...,NaN,NaN,NaN,NaN,State defined mine status: Expired. Reclamatio...,8,https://surface-mining.alabama.gov/,4223.617167,7.008642e+05,"MULTIPOLYGON (((-9540725.233 4143212.597, -954..."


In [56]:
mines.head(10)

,OBJECTID,permittee,company,coalmine_op_status,mine_name,permit_id,msha_id,state_tribe_mine_id,national_id,coal_bed_names,...,mountaintop,steep_slope,highwall,auger,comment,contact,information_link,shape_Length,shape_Area,geometry
0,1,HIGGINS COAL CO INC,HIGGINS COAL CO INC,7,MINE #50,2404,NaN,NaN,TN2404,NaN,...,NaN,NaN,NaN,NaN,NaN,4,https://www.osmre.gov/,1220.769913,1.354382e+04,"MULTIPOLYGON (((-9515951.177 4212750.257, -951..."
1,2,MIDNIGHT COAL CO,MIDNIGHT COAL CO,7,MINE #1,2405,NaN,NaN,TN2405,NaN,...,NaN,NaN,NaN,NaN,NaN,4,https://www.osmre.gov/,860.404674,1.678249e+04,"MULTIPOLYGON (((-9516024.476 4212708.540, -951..."
2,3,META ELKHORN MINING CO INC,META ELKHORN MINING CO INC,7,AREA 11,2756,NaN,NaN,TN2756,NaN,...,NaN,NaN,NaN,NaN,NaN,4,https://www.osmre.gov/,7219.650616,1.689153e+06,"MULTIPOLYGON (((-9459511.615 4347543.957, -945..."
3,4,JORDAN BROTHERS COAL CO INC,JORDAN BROTHERS COAL CO INC,7,TIPPLE #1,2497,NaN,NaN,TN2497,NaN,...,NaN,NaN,NaN,NaN,NaN,4,https://www.osmre.gov/,688.042643,2.188255e+04,"MULTIPOLYGON (((-9372128.011 4337061.595, -937..."
4,5,DOUBLE Q INC,DOUBLE Q INC,7,ANGELA #1,2284023,NaN,NaN,TN2284023,NaN,...,NaN,NaN,NaN,NaN,NaN,4,https://www.osmre.gov/,7994.318732,8.636343e+04,"MULTIPOLYGON (((-9344712.695 4372890.512, -934..."
5,6,MONARCH COALS INC,MONARCH COALS INC,7,MINE #2,2757,NaN,NaN,TN2757,NaN,...,NaN,NaN,NaN,NaN,NaN,4,https://www.osmre.gov/,2213.034790,2.407569e+04,"MULTIPOLYGON (((-9384008.258 4337104.160, -938..."
6,7,TENNESSEE MINING INC,TENNESSEE MINING INC,7,DEEP MINE #1,3043,NaN,NaN,TN3043,NaN,...,NaN,NaN,NaN,NaN,NaN,4,https://www.osmre.gov/,1900.028725,7.533373e+04,"MULTIPOLYGON (((-9401053.569 4332218.995, -940..."
7,8,TENNESSEE MINING INC,TENNESSEE MINING INC,7,AREA 8 RECLAIMED,3044,NaN,NaN,TN3044,NaN,...,NaN,NaN,NaN,NaN,NaN,4,https://www.osmre.gov/,10440.718347,5.073089e+05,"MULTIPOLYGON (((-9403465.426 4329370.524, -940..."
8,9,APPOLO FUELS INC,APPOLO FUELS INC,7,HORSESHOE MOUNTAIN,3045,NaN,NaN,TN3045,NaN,...,NaN,NaN,NaN,NaN,NaN,4,https://www.osmre.gov/,19573.642109,8.678622e+05,"MULTIPOLYGON (((-9335277.029 4375411.183, -933..."
9,10,WEST COAL CORP,WEST COAL CORP,7,NEWTOWN TIPPLE,2504,NaN,NaN,TN2504,NaN,...,NaN,NaN,NaN,NaN,NaN,4,https://www.osmre.gov/,1063.955436,1.514327e+04,"MULTIPOLYGON (((-9400574.571 4356368.847, -940..."


In [57]:

# Read data using GeoPandas
print("Data dimensions: {}".format(mines.shape))
mines = mines.head(10)

Data dimensions: (25124, 31)


In [58]:
mines = mines.drop(columns=['company'])

In [59]:
mines = mines.drop(columns=['permittee'])
mines = mines.drop(columns=['mine_name'])
mines = mines.drop(columns=['msha_id'])
mines = mines.drop(columns=['state_tribe_mine_id'])
mines = mines.drop(columns=['mountaintop'])
mines = mines.drop(columns=['steep_slope'])
mines = mines.drop(columns=['highwall'])
mines = mines.drop(columns=['auger'])
mines = mines.drop(columns=['comment'])
mines = mines.drop(columns=['coal_bed_names'])
mines = mines.drop(columns=['date_of_map'])
mines = mines.drop(columns=['inspectable_unit_status'])
mines = mines.drop(columns=['permit_approval_date'])
mines = mines.drop(columns=['edit_date'])
mines = mines.drop(columns=['area_mine'])
mines = mines.drop(columns=['contour'])
mines = mines.drop(columns=['permit_application_type'])
mines = mines.drop(columns=['reported_area'])
mines = mines.drop(columns=['permit_application_date'])
mines = mines.drop(columns=['information_link'])
mines = mines.drop(columns=['post_smcra'])


mines = mines.rename(columns={'OBJECTID': 'shapeName'})

In [60]:
mines

,shapeName,coalmine_op_status,permit_id,national_id,calculated_area,contact,shape_Length,shape_Area,geometry
0,1,7,2404,TN2404,2.220938,4,1220.769913,1.354382e+04,"MULTIPOLYGON (((-9515951.177 4212750.257, -951..."
1,2,7,2405,TN2405,2.752046,4,860.404674,1.678249e+04,"MULTIPOLYGON (((-9516024.476 4212708.540, -951..."
2,3,7,2756,TN2756,270.299198,4,7219.650616,1.689153e+06,"MULTIPOLYGON (((-9459511.615 4347543.957, -945..."
3,4,7,2497,TN2497,3.507990,4,688.042643,2.188255e+04,"MULTIPOLYGON (((-9372128.011 4337061.595, -937..."
4,5,7,2284023,TN2284023,13.751147,4,7994.318732,8.636343e+04,"MULTIPOLYGON (((-9344712.695 4372890.512, -934..."
5,6,7,2757,TN2757,3.859603,4,2213.034790,2.407569e+04,"MULTIPOLYGON (((-9384008.258 4337104.160, -938..."
6,7,7,3043,TN3043,12.087611,4,1900.028725,7.533373e+04,"MULTIPOLYGON (((-9401053.569 4332218.995, -940..."
7,8,7,3044,TN3044,81.452394,4,10440.718347,5.073089e+05,"MULTIPOLYGON (((-9403465.426 4329370.524, -940..."
8,9,7,3045,TN3045,138.177280,4,19573.642109,8.678622e+05,"MULTIPOLYGON (((-9335277.029 4375411.183, -933..."
9,10,7,2504,TN2504,2.419121,4,1063.955436,1.514327e+04,"MULTIPOLYGON (((-9400574.571 4356368.847, -940..."


In [61]:
# Get the shape geometry for Tennesssee mines
region = eec.gdfToFc(mines)

In [62]:
df = pd.DataFrame()

In [63]:
mines_exploded=mines.explode()
mines_exploded

<ipython-input-63-9f8c0177ef78>:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  mines_exploded=mines.explode()


shapeName  coalmine_op_status permit_id national_id  calculated_area  \
0 0          1                   7      2404      TN2404         2.220938   
1 0          2                   7      2405      TN2405         2.752046   
2 0          3                   7      2756      TN2756       270.299198   
3 0          4                   7      2497      TN2497         3.507990   
  1          4                   7      2497      TN2497         3.507990   
4 0          5                   7   2284023   TN2284023        13.751147   
  1          5                   7   2284023   TN2284023        13.751147   
  2          5                   7   2284023   TN2284023        13.751147   
  3          5                   7   2284023   TN2284023        13.751147   
5 0          6                   7      2757      TN2757         3.859603   
  1          6                   7      2757      TN2757         3.859603   
6 0          7                   7      3043      TN3043        12.087611   
7 0          8                   7      3044      TN3044        81.452394   
  1          8                   7      3044      TN3044        81.452394   
8 0          9                   7      3045      TN3045       138.177280   
9 0         10                   7      2504      TN2504         2.419121   
  1         10                   7      2504      TN2504         2.419121   

     contact  shape_Length    shape_Area  \
0 0        4   1220.769913  1.354382e+04   
1 0        4    860.404674  1.678249e+04   
2 0        4   7219.650616  1.689153e+06   
3 0        4    688.042643  2.188255e+04   
  1        4    688.042643  2.188255e+04   
4 0        4   7994.318732  8.636343e+04   
  1        4   7994.318732  8.636343e+04   
  2        4   7994.318732  8.636343e+04   
  3        4   7994.318732  8.636343e+04   
5 0        4   2213.034790  2.407569e+04   
  1        4   2213.034790  2.407569e+04   
6 0        4   1900.028725  7.533373e+04   
7 0        4  10440.718347  5.073089e+05   
  1        4  10440.718347  5.073089e+05   
8 0        4  19573.642109  8.678622e+05   
9 0        4   1063.955436  1.514327e+04   
  1        4   1063.955436  1.514327e+04   

                                              geometry  
0 0  POLYGON ((-9515951.177 4212750.257, -9515928.8...  
1 0  POLYGON ((-9516024.476 4212708.540, -9516019.2...  
2 0  POLYGON ((-9459511.615 4347543.957, -9459509.4...  
3 0  POLYGON ((-9372128.011 4337061.595, -9372159.8...  
  1  POLYGON ((-9372169.449 4337098.162, -9372176.4...  
4 0  POLYGON ((-9344712.695 4372890.512, -9344730.1...  
  1  POLYGON ((-9344856.309 4373121.701, -9344853.7...  
  2  POLYGON ((-9344923.815 4375220.728, -9344944.3...  
  3  POLYGON ((-9344979.198 4376074.780, -9344991.6...  
5 0  POLYGON ((-9384008.258 4337104.160, -9384016.8...  
  1  POLYGON ((-9383978.169 4337353.417, -9383972.0...  
6 0  POLYGON ((-9401053.569 4332218.995, -9401057.0...  
7 0  POLYGON ((-9403465.426 4329370.524, -9403511.6...  
  1  POLYGON ((-9403314.278 4329115.953, -9403331.3...  
8 0  POLYGON ((-9335277.029 4375411.183, -9335284.9...  
9 0  POLYGON ((-9400574.571 4356368.847, -9400575.3...  
  1  POLYGON ((-9400763.612 4356459.844, -9400747.7...

In [64]:
region

In [65]:
i=1

# Initial date of interest (inclusive).
i_date = '2021-01-01'
# Final date of interest (exclusive).
f_date = '2021-01-15'

# Extract the coordinates from the polygon
polygon = mines_exploded['geometry'].values[i]

#needs adjusting for different geometry types
coordinates = polygon.exterior.coords[:]
projected_polygon_shape = np.asarray(coordinates)

# Create an ee.Geometry object from the coordinates
polygon = ee.Geometry.MultiPolygon(coordinates)

# Get dynamic world image collection by asset ID
dc = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterBounds(region).filterDate(i_date, f_date)

# get band names, equal to column namesbandnames = dc.first().bandNames().getInfo()[:-1]

# Set the maxError parameter
max_error = 1  # Specify your desired maxError value

# Project the geometry coordinates
projected_polygon = polygon.transform('EPSG:4326', max_error)

#decrease scale to get more rows of data/granularity
ac = dc.getRegion(projected_polygon, scale=50).getInfo()
df_constructor =pd.DataFrame(ac)

df_constructor.columns = df_constructor.iloc[0]
df_constructor = df_constructor.tail(-1)
df_constructor['company'] = mines_exploded['company'].values[i]
df_constructor['shapeName'] = mines_exploded['shapeName'].values[i]
df_constructor['start_date'] = i_date
df_constructor['end_Date'] = f_date
df = df.append(df_constructor)


EEException: ignored

In [13]:
import topojson as tp
topo = tp.Topology(mines_exploded, prequantize=False)
mines_exploded = topo.toposimplify(2).to_gdf()

ModuleNotFoundError: ignored

In [ ]:
i=0

# Initial date of interest (inclusive).
i_date = '2021-01-01'
# Final date of interest (exclusive).
f_date = '2021-01-15'

df_contructor = []
while i<=len(mines_exploded):
 try:
    # Extract the coordinates from the polygon
    polygon = gdf_simplified['geometry'].values[i]

    #needs adjusting for different geometry types
    coordinates = polygon.exterior.coords[:]
    projected_polygon_shape = np.asarray(coordinates)

    projected_polygon_shape = projected_polygon_shape[z]
    bound_object = ee.Geometry.MultiPolygon(coordinates)
    bound_object = bound_object.bounds()

    # Create an ee.Geometry object from the coordinates
    polygon = ee.Geometry.MultiPolygon(coordinates)

    # Get dynamic world image collection by asset ID
    dc = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate(i_date, f_date)

    # get band names, equal to column namesbandnames = dc.first().bandNames().getInfo()[:-1]

    # Set the maxError parameter
    max_error = 1  # Specify your desired maxError value

    # Project the geometry coordinates
    projected_polygon = polygon.transform('EPSG:4326', max_error)

    #decrease scale to get more rows of data/granularity
    ac = dc.getRegion(projected_polygon, scale=50000000000 ).getInfo()
    df_constructor =pd.DataFrame(ac)

    df_constructor.columns = df_constructor.iloc[0]
    df_constructor = df_constructor.tail(-1)
    df_constructor['company'] = mines_exploded['company'].values[i]
    df_constructor['shapeName'] = mines_exploded['shapeName'].values[i]
    df_constructor['start_date'] = i_date
    df_constructor['end_Date'] = f_date
    df = df.append(df_constructor)


        print(mines_exploded['company'].values[i])
        i+=1
 except:
          i+=1
          pass



IndentationError: ignored

In [ ]:
df['company'].unique()

In [ ]:
df

In [ ]:
len(df['company'].unique())

In [ ]:
len(mines_exploded['company'].unique())